## Imports

In [165]:
#Branch Testing

In [166]:
import pandas as pd
import os
import numpy as np
from pyvis.network import Network
from neo4j import GraphDatabase


## Functions 

### Find Each Table's primary key

In [167]:
def pk_finder(name, df):
    for i in range(len(df.columns)):
        col = df.iloc[:,i]
        #if(col.size == col.drop_duplicates().size):
        if(col.nunique() == col.size):
            pk[name] = df.columns[i] 
            break
        

### Find Each Table's foreign key


In [168]:
def fk_finder(name, df):
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        col = df.iloc[:,i]
        if(list(df.columns)[i] not in pk):
            flag = True
            for j in pk: #loops on the key j = table names
                if j != name:
                    primary_key_column = All_dfs[j][(pk[j])]
                    if primary_key_column.dtype == col.dtype:
                        status = col.isin(primary_key_column)
                        if status[status== False].size == 0:
                            fk[name][col_name] = j 
                            ref_in[j].add(name)
                            break

### Assign Tables to nodes or edges

In [169]:
def nodes_edges(fk):
    for f in fk:
        if len(fk[f]) == 2:
            inner_dict = list(fk[f])
            edges[f] = fk[f]
        elif len(fk[f]) == 1:
            properties[f] = fk[f]
        else:
            nodes[f] = All_dfs[f]
#         else:
#             if len(fk[f]) == 1:
#                 properties[f] = fk[f]
            
#             nodes[f] = All_dfs[f]
    # for f in fk:
    #     if((f not in edges) ):
    #         nodes[f] = All_dfs[f]
    #         #nodes.append(f)
        

### Create Nodes Table

In [170]:
def add_nodes(nodes, nodesTable):
    for n in range(len(nodes)):
        for index, row in nodes[list(nodes.keys())[n]].iterrows():
            column_names = list(nodes[list(nodes.keys())[n]].columns) #get column names
            att = {}
            for i in range(1, len(column_names)):
                att[column_names[i]] = nodes[list(nodes.keys())[n]].iloc[index, i]
            newRow = [{'Label': list(nodes.keys())[n] , 'ID': nodes[list(nodes.keys())[n]].iloc[index,0] , 'Attributes': att}]
            tmp = pd.DataFrame(newRow)
            nodesTable = pd.concat([nodesTable, tmp], ignore_index=True)
    return nodesTable


### Create Edges Table

In [171]:
def add_edges(edges,edgesTable,pk,fk,ref_in):
    for e in edges:
        df = All_dfs[e] #df of the cur edge
        for r in range(len(df)):
            label = list(edges[e].keys())
            from_id = nodesTable[(nodesTable["Label"] == edges[e][label[0]]) & (df.loc[r,label[0]] == nodesTable["ID"])]
            from_id = from_id.index[0]
            to_id = nodesTable[(nodesTable["Label"] == edges[e][label[1]]) & (df.loc[r,label[1]] == nodesTable["ID"])]
            to_id = to_id.index[0]
            pk_col = pk[e]
#             primary_key = df[pk_col].iloc[r]
            primary_key = df.loc[r,pk_col]
            att = convert_prop(ref_in,e,primary_key,fk)
            newRow = [{'From_Node_ID': from_id, 'To_Node_ID': to_id, 'order/service' : att}]
            tmp = pd.DataFrame(newRow)
            edgesTable = pd.concat([edgesTable, tmp], ignore_index=True)
    return edgesTable

### Properties of the Edge

In [172]:
def convert_prop(ref_in,edge_key, pk_value,fk):
    for referenced_table_name in ref_in[edge_key]:
        referenced_table = All_dfs[referenced_table_name]
        fk_of_ref_table = fk[referenced_table_name]
        for foreign_key in fk_of_ref_table:
            if fk_of_ref_table[foreign_key] == edge_key:
#                 referenced_table[referenced_table[foreign_key]== id]
                all_occurances_df = referenced_table[referenced_table[foreign_key]== pk_value].drop([foreign_key],axis=1)
                return all_occurances_df.to_dict('records')
                

### generate list of n colors

In [173]:
import random
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
# get_colors(5) # sample return:  ['#8af5da', '#fbc08c', '#b741d0', '#e599f1', '#bbcb59', '#a2a6c0']

### Draw Pyvis graph

In [174]:
def draw_graph_pyvis(nodes_table,edges_table):

    nodes_name = [(x[1]+'_'+str(x[0])).capitalize() for x in zip(nodes_table.ID,nodes_table.Label)]
    indices = list(map(lambda x: int(x),list(nodes_table.index)))
    nodes_attributes = list(map(lambda x: str(x).replace(',',"\n"),list(nodes_table.Attributes)))
    nodes_tables = list(nodes_df.Label.unique())
    colors = get_colors(len(nodes_name))
    nodes_color = list(map(lambda x: colors[nodes_tables.index(x)],list(nodes_table.Label)))
    minium_weight: int = 0,
    repulsion: int = 100,
    spring_length=200,
    
    g = Network(width='100%')
#     g.barnes_hut()
    g.add_nodes(indices,
                title= nodes_attributes,
                label=nodes_name,
                color=nodes_color,
                )
    for e in range(len(edges_table)):
        src = int(edges_table.iloc[e]["From_Node_ID"])
        dst = int(edges_table.iloc[e]["To_Node_ID"])
        g.add_edge(src,dst,weight=5)

    print("Number of Nodes: ",len(g.get_nodes()))
    print("Number of Edges: ",len(g.get_edges()))
    g.repulsion(repulsion, spring_length=spring_length)
    g.show_buttons(filter_=True)



    

    g.show('test.html')




### Draw Graph Neo4j

#### prepare satements

In [175]:
def draw_graph_Neo4j(nodes_table,edges_table):

    transaction_execution_commands = []
    # creating a statement for each node (row in node_table)
    for i, node in nodes_table.iterrows():
        label = node["Label"]
        index = i
        ID = node["ID"]
        attributes = node["Attributes"]
        neo4j_create_statemenet = Create_Statement(label,i,ID,attributes)
#         print(neo4j_create_statemenet)
        transaction_execution_commands.append(neo4j_create_statemenet)

     
    execute_transactions(transaction_execution_commands)
 

#### Connect and execute statements

In [176]:
def execute_transactions(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    session = data_base_connection.session()    
    for i in transaction_execution_commands:
        session.run(i)

 

#### Create a "Create statement for nodes"

In [177]:
def Create_Statement(label,index,ID,attributes):
    return f"CREATE (x:{label} {'{'}name: {get_name(label,ID)},index:{index},ID:{ID} ,{destrucure_dict(attributes)} {'}'})"


In [178]:
def get_name(label,ID):
    return '\"' + label.capitalize()+'_'+str(ID)+'\"'

#### prepare attributes for create statement

In [179]:
def destrucure_dict(dic):
    s = ""
    for key in dic:
        value = dic[key]
        if isinstance (value,str):
            value = "\"" + value +"\""
        s=s + str(key) + ":" + str(value)+","
    return s[:-1]

## Global Variables

In [180]:
# {Table name: df}
All_dfs = {}
# {Table name: Table's Primary key}
pk = {}
# {Table name: {Column Name: Referenced Table}}
fk = {}
# {Table name: set(Tables that references the table)}
ref_in = {}
# {Table name: df}
nodes = {}
# {Table name: {Column Name: Referenced Table}}
edges = {}
# _
properties = {}
# Nodes_df
nodesTable = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# Edges_df
edgesTable = pd.DataFrame(columns = ['From_Node_ID', 'To_Node_ID', 'order/service'])
# Properties_df
propertiesTable = pd.DataFrame(columns = [['index','From', 'Label', 'Attributes']])
#Label = ['facilities'/ 'order' / 'service']


# Nodes
# Edges ['From_Node_ID', 'To_Node_ID']
# Properties ['Label', 'ID', 'Attributes']
# (^Nodes in graph)
# Relationship ["Relation_name","from","to"]


nodes_df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
edges_df = pd.DataFrame(columns=['From','To'])


## Reading Data Set

 creating a dictionary where
 key: first word of the table name 
 value: df 

In [181]:
import os
path_of_the_directory = './DataSet/'
ext = ('.csv')
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        print(file) 
        temp = (file.replace("_"," ").replace("."," ").split(" ")[0].lower())
        All_dfs[temp] = pd.read_csv(path_of_the_directory+file)
    else:
        continue

InternalTransactions.csv
InternalShipments_data.csv
facilities_data.csv
ExternalShipments_data.csv
InternalServices_data.csv
warehouses_data.csv
InternalOrders_data.csv
customer_data.csv
Products_data.csv
Supplier_data.csv
ExternalTransactions.csv
ExternalOrders_data.csv
Manufacturing_data.csv
ExternalServices_data.csv
Retailer_data.csv


### Splitting Shipments tables

In [182]:
SRIntShip = All_dfs["internalshipments"].query('from_to_where == "SR"')
SRIntShip = SRIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SRIntShip
RCExtShip = All_dfs["externalshipments"].query('from_to_where == "RC"')
RCExtShip = RCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#RCExtShip
SSIntShip = All_dfs["internalshipments"].query('from_to_where == "SS"')
SSIntShip = SSIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SSIntShip
SCExtShip = All_dfs["externalshipments"].query('from_to_where == "SC"')
SCExtShip = SCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SCExtShip
All_dfs["RCExtShip".lower()] = RCExtShip
All_dfs["SCExtShip".lower()] = SCExtShip
All_dfs["SRIntShip".lower()] = SRIntShip
All_dfs["SSIntShip".lower()] = SSIntShip

### Splitting Orders Tables

In [183]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]
All_dfs["rcextorders"] = All_dfs["externalorders"][All_dfs["externalorders"].ExtShip_id.isin(filter_list)].reset_index(drop=True)
filter_list = All_dfs["scextship"]["ExtShip_id"]
All_dfs["scextorders"] = All_dfs["externalorders"][All_dfs["externalorders"].ExtShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["scextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["srintship"]["IntShip_id"]
All_dfs["srintorders"] = All_dfs["internalorders"][All_dfs["internalorders"].IntShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["srintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["ssintship"]["IntShip_id"]
All_dfs["ssintorders"] = All_dfs["internalorders"][All_dfs["internalorders"].IntShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["ssintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)

### Removing Redundant Tables

In [184]:
All_dfs.pop("internalshipments")
All_dfs.pop("externalshipments")
All_dfs.pop("externalorders")
All_dfs.pop("internalorders")
All_dfs.keys()

dict_keys(['internaltransactions', 'facilities', 'internalservices', 'warehouses', 'customer', 'products', 'supplier', 'externaltransactions', 'manufacturing', 'externalservices', 'retailer', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Execlude Tables

In [185]:
All_dfs = {key: value for key, value in All_dfs.items() 
           if key not in ["products","warehouses","manufacturing"]}
All_dfs.keys()

dict_keys(['internaltransactions', 'facilities', 'internalservices', 'customer', 'supplier', 'externaltransactions', 'externalservices', 'retailer', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Initialize fk and ref_in

In [186]:
table_name = list(All_dfs.keys())
for table_name in All_dfs:
    fk[table_name] = {}
    ref_in[table_name] = set()

### Get Primary key for each table

In [187]:
for t in All_dfs:
    pk_finder(t,All_dfs[t])
pk

{'internaltransactions': 'IntTran_id',
 'facilities': 'fac_id',
 'internalservices': 'IntServ_id',
 'customer': 'cust_id',
 'supplier': 'supp_id',
 'externaltransactions': 'ExtTran_id',
 'externalservices': 'ExtServ_id',
 'retailer': 'retailer_id',
 'rcextship': 'ExtShip_id',
 'scextship': 'ExtShip_id',
 'srintship': 'IntShip_id',
 'ssintship': 'IntShip_id',
 'rcextorders': 'ExtOrders_id',
 'scextorders': 'ExtOrders_id',
 'srintorders': 'IntOrders_id',
 'ssintorders': 'IntOrders_id'}

### Get Foreing key for each table

In [188]:
for t in All_dfs:
    fk_finder(t,All_dfs[t])
fk

{'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'customer': {},
 'supplier': {},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'retailer': {},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'},
 'ssintship': {'listSuppIds': 'supplier', 'factoryIds': 'supplier'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

In [189]:
ref_in

{'internaltransactions': {'internalservices'},
 'facilities': set(),
 'internalservices': set(),
 'customer': {'externaltransactions', 'rcextship', 'scextship'},
 'supplier': {'externaltransactions',
  'facilities',
  'internaltransactions',
  'scextship',
  'srintship',
  'ssintship'},
 'externaltransactions': {'externalservices'},
 'externalservices': set(),
 'retailer': {'rcextship', 'srintship'},
 'rcextship': {'rcextorders'},
 'scextship': {'scextorders'},
 'srintship': {'srintorders'},
 'ssintship': {'ssintorders'},
 'rcextorders': set(),
 'scextorders': set(),
 'srintorders': set(),
 'ssintorders': set()}

### Determine nodes and edges

In [190]:
nodes_edges(fk)

In [191]:
nodes.keys()

dict_keys(['customer', 'supplier', 'retailer'])

In [192]:
edges

{'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'},
 'ssintship': {'listSuppIds': 'supplier', 'factoryIds': 'supplier'}}

In [193]:
properties

{'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

### Create NodesTable

In [194]:
nodesTable = add_nodes(nodes,nodesTable)
nodesTable

Label     ID                                         Attributes
0    customer  33736  {'gender': 'F', 'first_name': 'Nadia', 'last_n...
1    customer  42959  {'gender': 'M', 'first_name': 'Eric', 'last_na...
2    customer  50653  {'gender': 'F', 'first_name': 'Alessia', 'last...
3    customer  82485  {'gender': 'M', 'first_name': 'Alexei', 'last_...
4    customer  56609  {'gender': 'F', 'first_name': 'Brenda', 'last_...
..        ...    ...                                                ...
595  retailer  86308  {'country': 'Ukraine', 'city_name': 'Chernihiv...
596  retailer  18464  {'country': 'Haiti', 'city_name': 'Grandans', ...
597  retailer  57039  {'country': 'Lao People's Democratic Republic'...
598  retailer  33250  {'country': 'South Sudan', 'city_name': 'Lakes...
599  retailer  21789  {'country': 'Cyprus', 'city_name': 'Ammochosto...

[600 rows x 3 columns]

### Create Edges Table

In [195]:
edgesTable = add_edges(edges, edgesTable,pk,fk,ref_in)
edgesTable

From_Node_ID To_Node_ID                                      order/service
0            269        309  [{'IntServ_id': 2070, 'placed_when': '2002-01-...
1            324        443  [{'IntServ_id': 1165, 'placed_when': '2000-08-...
2            403        396  [{'IntServ_id': 4897, 'placed_when': '2009-04-...
3            353        409                                                 []
4            247        412                                                 []
..           ...        ...                                                ...
395          309        198  [{'IntOrders_id': 2751, 'quantity': 734, 'plac...
396          171        321  [{'IntOrders_id': 1084, 'quantity': 784, 'plac...
397          193        201  [{'IntOrders_id': 4939, 'quantity': 819, 'plac...
398          171        244  [{'IntOrders_id': 6448, 'quantity': 712, 'plac...
399          156        446  [{'IntOrders_id': 5094, 'quantity': 255, 'plac...

[400 rows x 3 columns]

In [196]:
# nodes_df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# edges_df = pd.DataFrame(columns=['From','To'])
all_nodes = list(nodes.keys())
for n in range(len(nodes)):
      for index, row in nodes[all_nodes[n]].iterrows():
          column_names = list(nodes[all_nodes[n]].columns) #get column names
          att = {}
          for i in range(1, len(column_names)):
              att[column_names[i]] = nodes[all_nodes[n]].iloc[index, i]
          newRow = [{'Label': all_nodes[n] , 'ID': nodes[all_nodes[n]].iloc[index,0] , 'Attributes': att}]
          tmp = pd.DataFrame(newRow)
          nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)

In [197]:
nodes_df.iloc[50:]

Label     ID                                         Attributes
50   customer  11346  {'gender': 'M', 'first_name': 'Mitchell', 'las...
51   customer  78650  {'gender': 'M', 'first_name': 'Mohamed', 'last...
52   customer  87252  {'gender': 'M', 'first_name': 'Robert', 'last_...
53   customer  24338  {'gender': 'M', 'first_name': 'Sante', 'last_n...
54   customer  18213  {'gender': 'M', 'first_name': 'Friedo', 'last_...
..        ...    ...                                                ...
595  retailer  86308  {'country': 'Ukraine', 'city_name': 'Chernihiv...
596  retailer  18464  {'country': 'Haiti', 'city_name': 'Grandans', ...
597  retailer  57039  {'country': 'Lao People's Democratic Republic'...
598  retailer  33250  {'country': 'South Sudan', 'city_name': 'Lakes...
599  retailer  21789  {'country': 'Cyprus', 'city_name': 'Ammochosto...

[550 rows x 3 columns]

In [198]:
all_edges = list(edges.keys())
for n in range(len(edges)):
      Edge_name = all_edges[n]
      
      foreign_keys = list(fk[Edge_name].keys())

      from_col = foreign_keys[0]
      from_table_name = fk[Edge_name][from_col]
      from_df = All_dfs[from_table_name]
      from_df_pk = pk[from_table_name]
      
      to_col = foreign_keys[-1]
      to_table_name = fk[Edge_name][to_col]
      to_df = All_dfs[to_table_name]
      to_df_pk = pk[to_table_name]
      
      column_names = list(All_dfs[Edge_name].columns) #get column names
      for index, _ in All_dfs[Edge_name].iterrows():
          att = {}
          from_ref_id,to_ref_id = None, None
          
          
          for i in range(1, len(column_names)):
            column_name = column_names[i]
            
            
            if column_name not in foreign_keys:
              att[column_name] = All_dfs[Edge_name].iloc[index, i]
          
            else:
              reference_id = All_dfs[Edge_name].iloc[index, i]
              if column_name == from_col:
                # from_ref_id = from_df[from_df[from_df_pk] == reference_id].index[0]
                from_ref_id = reference_id
              else:
                # to_ref_id = to_df[to_df[to_df_pk] == reference_id].index[0]
                to_ref_id = reference_id

          # Adding new entry to node tabel
          newRow = [{'Label': Edge_name , 'ID': All_dfs[Edge_name].iloc[index,0] , 'Attributes': att}]
          tmp = pd.DataFrame(newRow)
          nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)
          edge_node_index = len(nodes_df)-1
          # print(nodes_df.iloc[len(nodes_df)-1],All_dfs[Edge_name].iloc[index,0] )

          # creating two edges, one from the from_node to the edge node and one from edge node to to_node
          from_node_id = nodes_df[(nodes_df['Label']==from_table_name) &  (nodes_df['ID']==from_ref_id)].index[0]
          to_node_id =  nodes_df[(nodes_df['Label']==to_table_name) &  (nodes_df['ID']==to_ref_id)].index[0] 
          
          # from ---> edge 
          new_from_edge_row = [{'From': from_node_id , 'To':edge_node_index }]
          tmp = pd.DataFrame(new_from_edge_row)
          edges_df = pd.concat([edges_df, tmp], ignore_index=True)
          # edge --->to
          new_to_edge_row = [{'From': edge_node_index , 'To':  to_node_id}]
          tmp = pd.DataFrame(new_to_edge_row)
          edges_df = pd.concat([edges_df, tmp], ignore_index=True)



In [199]:
edges_df

From   To
0    269  600
1    600  309
2    324  601
3    601  443
4    403  602
..   ...  ...
795  997  201
796  171  998
797  998  244
798  156  999
799  999  446

[800 rows x 2 columns]

In [200]:
all_properties = list(properties.keys())
for property_name in all_properties:
  property_df = All_dfs[property_name]
      
  foreign_keys = list(fk[property_name].keys())

  fk_col = foreign_keys[0]
  referenced_table_name = fk[property_name][fk_col]
  referenced_table_df = All_dfs[referenced_table_name]
  referenced_table_pk = pk[referenced_table_name]
  
  column_names = list(property_df.columns) #get column names
  
  for index, _ in property_df.iterrows():    
      att = {}
      reference_id = None;      
      
      for i in range(1, len(column_names)):
        column_name = column_names[i]
        
        
        if column_name not in foreign_keys:
          att[column_name] = property_df.iloc[index, i]
      
        else:
          # capturing foreign key value
          reference_id = property_df.iloc[index, i]

      # Adding new entry to node tabel
      newRow = [{'Label': property_name , 'ID': property_df.iloc[index,0] , 'Attributes': att}]
      tmp = pd.DataFrame(newRow)
      nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)
      property_node_index = len(nodes_df)-1
      # print(nodes_df.iloc[len(nodes_df)-1],property_df.iloc[index,0] )

      # creating two edges, one from the from_node to the edge node and one from edge node to to_node
      referenced_node_id = nodes_df[(nodes_df['Label']==referenced_table_name) &  (nodes_df['ID']==reference_id)].index[0] 
      
      new_property_edge_row = [{'From': referenced_node_id , 'To':  property_node_index}]
      tmp = pd.DataFrame(new_property_edge_row)
      edges_df = pd.concat([edges_df, tmp], ignore_index=True)



In [201]:
edges_df

From    To
0     269   600
1     600   309
2     324   601
3     601   443
4     403   602
...   ...   ...
3195  951  3395
3196  992  3396
3197  970  3397
3198  992  3398
3199  977  3399

[3200 rows x 2 columns]

In [202]:
draw_graph_pyvis(nodesTable,edgesTable)

Number of Nodes:  600
Number of Edges:  400


In [203]:
sum_nodes = 0
sum_edges = 0
for key in All_dfs:
    table_size = len(All_dfs[key])
    print(key, table_size)
    sum_nodes += table_size
    if(key in all_edges):
        sum_edges += table_size*2
    if(key in all_properties ):
        sum_edges += table_size
print("Number Of Nodes: ",sum_nodes)
print("Number Of Edges: ",sum_edges)

internaltransactions 100
facilities 2000
internalservices 100
customer 150
supplier 300
externaltransactions 100
externalservices 100
retailer 150
rcextship 51
scextship 49
srintship 51
ssintship 49
rcextorders 52
scextorders 48
srintorders 54
ssintorders 46
Number Of Nodes:  3400
Number Of Edges:  3200


In [204]:
nodes_df

Label     ID                                         Attributes
0        customer  33736  {'gender': 'F', 'first_name': 'Nadia', 'last_n...
1        customer  42959  {'gender': 'M', 'first_name': 'Eric', 'last_na...
2        customer  50653  {'gender': 'F', 'first_name': 'Alessia', 'last...
3        customer  82485  {'gender': 'M', 'first_name': 'Alexei', 'last_...
4        customer  56609  {'gender': 'F', 'first_name': 'Brenda', 'last_...
...           ...    ...                                                ...
3395  ssintorders   7965  {'quantity': 259, 'placed_when': '2015-07-26',...
3396  ssintorders   3273  {'quantity': 414, 'placed_when': '2003-03-01',...
3397  ssintorders   5047  {'quantity': 389, 'placed_when': '1973-11-01',...
3398  ssintorders   5886  {'quantity': 106, 'placed_when': '2005-06-09',...
3399  ssintorders   8847  {'quantity': 525, 'placed_when': '2007-02-09',...

[3400 rows x 3 columns]

In [205]:
edges_df

From    To
0     269   600
1     600   309
2     324   601
3     601   443
4     403   602
...   ...   ...
3195  951  3395
3196  992  3396
3197  970  3397
3198  992  3398
3199  977  3399

[3200 rows x 2 columns]

In [206]:
import time
for property in range(len(list(properties.keys()))):
  table_name = "facilities"
  if(list(properties.keys())[property] == table_name):
    referencedTableName = properties[table_name][list(properties[table_name].keys())[0]]
    if(referencedTableName in nodes.keys()):
      print("Referenced Table is Node")
    elif(referencedTableName in edges.keys()):
      print("Referenced Table is Edge")
    referencedColName = list(properties[table_name].keys())[0]
    fac_df = All_dfs[list(properties.keys())[property]]
    time.sleep(1)
    for index in range(fac_df.shape[0]):
      fac = fac_df.iloc[index]
      fromId = fac[pk[list(properties.keys())[property]]]
      toId = fac[referencedColName]
      print(fromId)
      print(toId)
      print("---------")


Referenced Table is Node
694420
81114
---------
678955
81114
---------
719035
81114
---------
240432
81114
---------
385384
81114
---------
661839
81114
---------
263477
81114
---------
144639
81114
---------
858404
81114
---------
991077
81114
---------
915035
68177
---------
905177
68177
---------
701418
68177
---------
952688
68177
---------
161033
68177
---------
480583
68177
---------
909341
68177
---------
480533
68177
---------
699500
68177
---------
404028
68177
---------
957998
70869
---------
907715
70869
---------
790711
70869
---------
282126
70869
---------
399770
70869
---------
898663
70869
---------
577085
70869
---------
559237
70869
---------
711270
70869
---------
869533
70869
---------
739435
99101
---------
726402
99101
---------
950184
99101
---------
698173
99101
---------
268235
99101
---------
310760
99101
---------
159199
99101
---------
872449
99101
---------
170447
99101
---------
854146
99101
---------
223443
58047
---------
264966
58047
---------
114492
58

In [207]:
for property in range(len(list(properties.keys()))):
  table_name = 'facilities'
  if(list(properties.keys())[property] == table_name):
    referencedTableName = fk[table_name][list(properties[table_name].keys())[0]]
    if(referencedTableName in nodes.keys()):
      print("Referenced Table is Node")
    elif(referencedTableName in edges.keys()):
      print("Referenced Table is Edge")
    referencedColName = list(properties[table_name].keys())[0]
    fac_df = All_dfs[list(properties.keys())[property]]
    for index in range(fac_df.shape[0]):
      fac = fac_df.iloc[index]
      fromId = fac[pk[list(properties.keys())[property]]]
      toId = fac[referencedColName]
      print(fromId)
      print(toId)
      print("---------")

Referenced Table is Node
694420
81114
---------
678955
81114
---------
719035
81114
---------
240432
81114
---------
385384
81114
---------
661839
81114
---------
263477
81114
---------
144639
81114
---------
858404
81114
---------
991077
81114
---------
915035
68177
---------
905177
68177
---------
701418
68177
---------
952688
68177
---------
161033
68177
---------
480583
68177
---------
909341
68177
---------
480533
68177
---------
699500
68177
---------
404028
68177
---------
957998
70869
---------
907715
70869
---------
790711
70869
---------
282126
70869
---------
399770
70869
---------
898663
70869
---------
577085
70869
---------
559237
70869
---------
711270
70869
---------
869533
70869
---------
739435
99101
---------
726402
99101
---------
950184
99101
---------
698173
99101
---------
268235
99101
---------
310760
99101
---------
159199
99101
---------
872449
99101
---------
170447
99101
---------
854146
99101
---------
223443
58047
---------
264966
58047
---------
114492
58